# Kakao Arena 2회 대회 : 브런치 사용자를 위한 글 추천 대회
## 데이터 EDA ipython notebook

In [53]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re

In [54]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from platform import python_version

In [55]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = './usr/share/fonts/NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name, size=12)
plt.rcParams["figure.figsize"] = (20, 10)
register_matplotlib_converters()

In [56]:
import dill

In [57]:
directory = './data/dataset/arena/res/'

In [58]:
print("Python Version : " + python_version())

Python Version : 3.7.1


## 1. Data Read

### a. Magazine.json

In [59]:
magazine = pd.read_json(directory + 'magazine.json', lines=True)

In [60]:
magazine.shape

(27967, 2)

In [61]:
magazine.head()

,id,magazine_tag_list
0,38842,"[브런치북, 육아일기, 대화법, 들려주고픈이야기]"
1,11540,"[tea, food]"
2,11541,[food]
3,11546,"[브런치북, 일상, 시, 사람]"
4,11544,"[감성에세이, 노래, 음악에세이]"


### b. Metadata.json

In [62]:
metadata = pd.read_json(directory + 'metadata.json', lines=True)

In [63]:
metadata.shape

(643104, 9)

In [64]:
metadata.head(2)

,article_id,display_url,id,keyword_list,magazine_id,reg_ts,sub_title,title,user_id
0,782,https://brunch.co.kr/@bookdb/782,@bookdb_782,"[여행, 호주, 국립공원]",8982,1474944427000,세상 어디에도 없는 호주 Top 10,"사진으로 옮기기에도 아까운, 리치필드 국립공원",@bookdb
1,81,https://brunch.co.kr/@kohwang56/81,@kohwang56_81,"[목련꽃, 아지랑이, 동행]",12081,1463092749000,,[시] 서러운 봄,@kohwang56


### c. Users.json

In [65]:
users = pd.read_json(directory + '/users.json', lines=True)

In [66]:
users.shape

(310758, 3)

In [67]:
users.head()

,following_list,id,keyword_list
0,"[@perytail, @brunch]",#901985d8bc4c481805c4a4f911814c4a,[]
1,"[@holidaymemories, @wadiz, @sciforus, @dailydu...",#1fd89e9dcfa64b45020d9eaca54e0eed,[]
2,"[@commerceguy, @sunsutu, @kakao-it, @joohoonja...",#1d94baaea71a831e1f33e1c6bd126ed5,[]
3,"[@amberjeon48, @forsy20, @nemotokki, @hawann, ...",#04641c01892b12dc018b1410e4928c0d,[]
4,"[@dwcha7342, @iammento, @kakao-it, @dkam, @ant...",#65bcaff862aadff877e461f54187ab62,[]


### d. Read Files

In [68]:
read_file_lst = glob.glob('./data/dataset/arena/res/read/*')

In [69]:
exclude_file_lst = ['read.tar']

In [ ]:
read_df_lst = []
for f in read_file_lst:
    file_name = os.path.basename(f)
    if file_name in exclude_file_lst:
        print(file_name)
    else:
        df_temp = pd.read_csv(f, header=None, names=['raw'],engine='python')
        df_temp['dt'] = file_name[:8]
        df_temp['hr'] = file_name[8:10]
        df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0]
        df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip()
        read_df_lst.append(df_temp)

In [ ]:
read = pd.concat(read_df_lst)

In [ ]:
read.shape

In [ ]:
read.head()

#### 탐색하기 좋은 데이터 포맷으로 변경

In [ ]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(' ')))

In [ ]:
read_cnt_by_user = read['article_id'].str.split(' ').map(len)

In [ ]:
read_raw = pd.DataFrame({'dt': np.repeat(read['dt'], read_cnt_by_user),
                         'hr': np.repeat(read['hr'], read_cnt_by_user),
                         'user_id': np.repeat(read['user_id'], read_cnt_by_user),
                         'article_id': chainer(read['article_id'])})

In [ ]:
read_raw.shape

In [ ]:
read_raw.head()

In [ ]:
print("전체 데이터 건수:", read_raw.shape)
print("중복 소비를 제외한 데이터 건수:", read_raw[['user_id', 'article_id']].drop_duplicates().shape)
print("Unique 독자 수:", len(read_raw['user_id'].unique()))
print("소비된 Unique 글 수:", len(read_raw['article_id'].unique()))

# EDA

## 1. 글 수 

### a. metadata 전처리

In [ ]:
atc = metadata.copy()

In [ ]:
atc['reg_datetime'] = atc['reg_ts'].apply(lambda x : datetime.fromtimestamp(x/1000.0))
atc.loc[atc['reg_datetime'] == atc['reg_datetime'].min(), 'reg_datetime'] = datetime(2090, 12, 31)
atc['reg_dt'] = atc['reg_datetime'].dt.date
atc['type'] = atc['magazine_id'].apply(lambda x : '개인' if x == 0.0 else '매거진')

In [ ]:
# 컬럼명 변경
atc.columns = ['id', 'display_url', 'article_id', 'keyword_list', 'magazine_id', 'reg_ts', 'sub_title', 'title', 'author_id', 'reg_datetime', 'reg_dt', 'type']

In [ ]:
atc.head()

### b. 등록일자별 글 수

In [ ]:
atc_cnt_by_reg_dt = atc.groupby('reg_dt', as_index=False)['article_id'].count()

In [ ]:
sns.lineplot(data=atc_cnt_by_reg_dt[:-1], x='reg_dt', y='article_id', color='#49beb7')
plt.title('등록일자별 글 수')
plt.xlabel('글 등록일')
plt.ylabel('글 수')

## 2. 글 소비

### a. 글별 소비수 통계

In [ ]:
atc_read_cnt = read_raw[read_raw.article_id != ''].groupby('article_id')['user_id'].count()

In [ ]:
# 글별 소비수 통계
atc_read_cnt.describe()

In [ ]:
# 글별 소비수 85% 95% 기준 값
atc_read_cnt.quantile([0.85, 0.95])

In [ ]:
atc_read_cnt = atc_read_cnt.reset_index()
atc_read_cnt.columns = ['article_id', 'read_cnt']

In [ ]:
atc_read_cnt.tail()

In [ ]:
#metadata 결합
atc_read_cnt = pd.merge(atc_read_cnt, atc, how='left', left_on='article_id', right_on='article_id')

In [ ]:
atc_read_cnt.shape

In [ ]:
atc_read_cnt.tail()

In [ ]:
# metadata를 찾을 수 없는 소비 로그 제외
atc_read_cnt_nn = atc_read_cnt[atc_read_cnt['id'].notnull()]

In [ ]:
# metadata를 찾을 수 없는 로그를 제외한 후 글별 소비수 통계
atc_read_cnt_nn['read_cnt'].describe()

In [ ]:
# 글별 소비수 90% 95% 기준 값
atc_read_cnt_nn['read_cnt'].quantile([0.90, 0.95])

In [ ]:
# 소비수 기준 분류값
def get_class(x):
    if x >= 142:
        result = '5%'
    elif x >= 72:
        result = '10%'
    elif x >= 25:
        result = '25%'
    elif x >= 8:
        result = '50%'
    elif x >= 3:
        result = '75%'
    else:
        result = '100%'
    return result

In [ ]:
atc_read_cnt_nn['class'] = atc_read_cnt_nn['read_cnt'].map(get_class)

In [ ]:
atc_read_cnt_nn.head()

### b. 등록일자별 글 소비수

In [ ]:
lm = sns.scatterplot(data=atc_read_cnt_nn, x='reg_dt', y='read_cnt', color='#49beb7')
lm.set(xlim=(datetime.date(datetime(2015, 4, 1)), datetime.date(datetime(2019, 3, 30))))
plt.title('등록일자별 글 소비수')
plt.xlabel('글 등록일')
plt.ylabel('글 소비수')

- 두 개의 아티클이 글 소비수가 매우 높음 
- 그래프를 자세히 보기 위해 두 개의 아티클을 제외하고 다시 그려보자

In [ ]:
atc_read_cnt_nn.sort_values(by='read_cnt', ascending=False).head(2)

In [ ]:
# 특이값 2개 데이터를 제외
lm = sns.scatterplot(data=atc_read_cnt_nn[~atc_read_cnt_nn.article_id.isin(['@brunch_141', '@brunch_151'])], x='reg_dt', y='read_cnt', hue='type', alpha=0.5, palette=['#49beb7', '#ff5959'])
lm.set(xlim=(datetime.date(datetime(2015, 4, 1)), datetime.date(datetime(2019, 3, 30))))
plt.title('등록일자별 글 소비수')
plt.xlabel('글 등록일')
plt.ylabel('글 소비수')

## 3. 경과일에 따른 글 소비 변화

### a. 데이터 전처리

In [ ]:
off_data = pd.merge(read_raw, atc, how='inner', left_on='article_id', right_on='article_id')

In [ ]:
off_data.shape

In [ ]:
off_data.head()

In [ ]:
off_data.columns = ['read_dt', 'hr', 'user_id', 'article_id', 'article_seq', 'display_url',
                    'keyword_list', 'magazine_id', 'reg_ts', 'sub_title', 'title',
                    'author_id', 'reg_datetime', 'reg_dt', 'type']

In [ ]:
off_data = off_data[['read_dt', 'user_id', 'article_id', 'title', 'sub_title', 'author_id', 'reg_dt', 'type', 'display_url', 'keyword_list', 'magazine_id']]

In [ ]:
off_data['read_dt'] = pd.to_datetime(off_data['read_dt'], format='%Y%m%d')
off_data['reg_dt'] = pd.to_datetime(off_data['reg_dt'], format='%Y-%m-%d')
off_data['off_day'] = (off_data['read_dt'] - off_data['reg_dt']).dt.days

In [ ]:
# meatadata와 join
off_data = pd.merge(off_data, atc_read_cnt_nn[['article_id', 'read_cnt', 'class']], how='left', left_on='article_id', right_on='article_id')

In [ ]:
off_data.shape

In [ ]:
off_data.head()

In [ ]:
off_data_agg = off_data.groupby(['article_id', 'off_day', 'read_dt', 'reg_dt', 'title', 'author_id', 'type', 'display_url', 'magazine_id', 'class'], as_index=False)['user_id'].count()

In [ ]:
# 등록일자가 missing된 로그들
off_data_agg[off_data_agg.off_day < 0].head()

### b. 경과일에 따른 글 소비 변화

In [ ]:
# 경과일자별 소비수 총합
off_day_sum = off_data_agg[(off_data_agg['reg_dt'] >= datetime(2018, 10, 1)) & (off_data_agg.off_day >= 0)].groupby('off_day')['user_id'].sum()

In [ ]:
# 경과일자에 등록되어있는 아티클 수
reg_dt_cnt = off_data_agg[(off_data_agg['reg_dt'] >= datetime(2018, 10, 1)) & (off_data_agg.off_day >= 0)].groupby('reg_dt')['article_id'].nunique()

In [ ]:
off_day_avg = pd.concat([off_day_sum, reg_dt_cnt.cumsum().sort_index(ascending=False).reset_index()], axis=1)

In [ ]:
off_day_avg['off_avg'] = off_day_avg['user_id'] / off_day_avg['article_id']

In [ ]:
off_day_avg['percentile'] = off_day_avg['off_avg'] / off_day_avg['off_avg'].sum()

In [ ]:
off_day_avg['cum_per'] = off_day_avg['percentile'].cumsum()

In [ ]:
off_day_avg.head(10)

In [ ]:
sns.lineplot(data=off_day_avg, x=off_day_avg.index, y='off_avg', color='#49beb7')
plt.title('경과일에 따른 글 소비수 변화')
plt.xlabel('경과일')
plt.ylabel('평균 글 소비수')

## 4. 위클리 매거진

In [ ]:
magazine_34075 = atc_read_cnt_nn[atc_read_cnt_nn.magazine_id == 34075]

In [ ]:
# Weekly매거진 회사 체질이 아니라서요 (by 서메리) 글 목록
magazine_34075

In [ ]:
magazine_34075_read = off_data_agg[off_data_agg.magazine_id == 34075]

In [ ]:
magazine_34075_read.head()

In [ ]:
sns.lineplot(data=magazine_34075_read, x='read_dt', y='user_id', hue='title')
plt.title('위클리-회사 체질이 아니라서요(by 서메리)')
plt.xlabel('글 소비일')
plt.ylabel('글 소비수')

## 5. 유저

In [ ]:
# 유져별 방문일수, 글 소비수
grp_by_user = off_data.groupby('user_id').agg({'read_dt':['nunique', 'count']})

In [ ]:
grp_by_user = grp_by_user.reset_index()
grp_by_user.columns = ['user_id', 'visit_day_cnt', 'read_cnt']

In [ ]:
grp_by_user.head()

### a. 방문일수 기준 유저 통계

In [ ]:
grp_by_user['visit_day_cnt'].value_counts(normalize=True).head()

In [ ]:
grp_by_user['visit_day_cnt'].value_counts(normalize=True).sort_index()[74:].sum()

- 1~2회 방문 유저 전체 유저중 50%

- 75회 이상 방문하는유저 전체 유저중 약 1%

- 1~2일 방문유저를 신규유저 / 75일(Train 기간 절반 방문) 방문 유저를 단골유저로 구분

In [ ]:
low_visit_user = grp_by_user.loc[grp_by_user['visit_day_cnt'] <=2, 'user_id']

In [ ]:
upp_visit_user = grp_by_user.loc[grp_by_user['visit_day_cnt'] >=75, 'user_id']

In [ ]:
low_visit_user.shape, upp_visit_user.shape

In [ ]:
off_data_l = off_data[off_data.user_id.isin(low_visit_user)]
off_data_u = off_data[off_data.user_id.isin(upp_visit_user)]

In [ ]:
off_data.shape, off_data_l.shape, off_data_u.shape

In [ ]:
1083925/20905040, 5002114/20905040

- 신규 유저의 글 소비는 전체의 5%, 단골 유저의 글 소비는 전체의 24%

In [ ]:
off_data_l.groupby('user_id')['article_id'].count().mean(), off_data_l.groupby('user_id')['article_id'].count().std(), 

In [ ]:
off_data_u.groupby('user_id')['article_id'].count().mean(), off_data_u.groupby('user_id')['article_id'].count().std()

- 신규 유저의 평균 글 소비수는 7, 표준편차 14
- 단골 유저의 평균 글 소비수는 1827, 표준편차 2758

### b. 신규/단골 그룹내 인기 글

In [ ]:
off_data_l_rk = off_data_l.groupby(['article_id', 'title', 'reg_dt'])['user_id'].count().sort_values(ascending=False)
off_data_l_rk = off_data_l_rk.reset_index()
off_data_l_rk['rk'] = off_data_l_rk.index + 1

In [ ]:
off_data_u_rk = off_data_u.groupby(['article_id', 'title', 'reg_dt'])['user_id'].count().sort_values(ascending=False)
off_data_u_rk = off_data_u_rk.reset_index()
off_data_u_rk['rk'] = off_data_u_rk.index + 1

In [ ]:
# 신규 독자 소비수 상위 상위 10개 글
off_data_l_rk.head(10)

In [ ]:
# 단골 독자 소비수 상위 상위 10개 글
off_data_u_rk.head(10)

In [ ]:
off_data_lu_rk = pd.merge(off_data_l_rk, off_data_u_rk, how='outer', left_on = 'article_id', right_on='article_id')

In [ ]:
off_data_lu_rk['diff_rk'] = off_data_lu_rk['rk_x'] - off_data_lu_rk['rk_y'] 

In [ ]:
# 신규 독자 소비수 상위 50개 글 중 단골 독자의 순위 차가 많이 나는 글
off_data_lu_rk[off_data_lu_rk.rk_x <= 50].sort_values(by='diff_rk', ascending=True).head(10)

In [ ]:
# 단골 독자 소비수 상위 50개 글 중 신규 독자의 순위 차가 많이 나는 긂
off_data_lu_rk[off_data_lu_rk.rk_y <= 50].sort_values(by='diff_rk', ascending=False).head(10)

## 6. Following List

### a. Following List 통계

In [ ]:
# following_list가 있는 유저
following = users[users['following_list'].str.len() !=0 ]

In [ ]:
following.shape[0] / users.shape[0]

- 98%의 유저가 follow하는 작가가 있음

In [ ]:
following['author_cnt'] = users['following_list'].str.len()

In [ ]:
#평균 구독자수 
following['author_cnt'].mean()

- follow하고 있는 유저는 평균 9명의 작가를 구독 중

In [ ]:
following['following_list'] = following['following_list'].apply(lambda x: ' '.join(x))

In [ ]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(' ')))

In [ ]:
following_lens = following['following_list'].str.split(' ').map(len)

In [ ]:
following_raw = pd.DataFrame({'id': np.repeat(following['id'], following_lens),
                         'following_list': chainer(following['following_list'])})

In [ ]:
# 가장 구독하는 유저가 많은 작가 리스트
following_raw['following_list'].value_counts(ascending=False)[:20]

In [ ]:
following_read = pd.merge(off_data, following_raw, how='inner', left_on=['user_id', 'author_id'], right_on=['id', 'following_list'])

In [ ]:
following_read.shape[0] / off_data.shape[0]

- 전체 소비 데이터중 구독하고 있는 작가의 글 소비 비중이 35%

In [ ]:
following

In [ ]:
following['author_cnt'].sort_values()

In [ ]:
following['author_cnt'].describe()

In [ ]:
following['author_cnt'].sort_values().values[round(len(following['author_cnt'])*0.5)]

In [ ]:
following['author_cnt'].plot()

In [ ]:
following_sort=following['author_cnt'].sort_values()
plt.plot(range(len(following_sort)),following_sort)
plt.show()

In [ ]:
# 이거 어떻게 돌려..
following['author_cnt'].sort_values().plot()

# follow 수는 500명언더에 매우 집합해 있음